# Training classifiers on HOG Data
#### For extraction of features, see extractFeatures.ipynb

conda env- faus_dl, make sure to switch to python3 kernel

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import sklearn
from sklearn import datasets, svm, metrics
import bcolz
import os, sys
import datetime as dt

In [4]:
import pickle

In [26]:
DISFA_path='/media/amogh/Stuff/CMU/datasets/DISFA_data/'
DISFA_AU_path=DISFA_path+'ActionUnit_Labels/'
print(DISFA_AU_path)
Videos_right_path=DISFA_path+'Videos_RightCamera/'
print(Videos_right_path)

/media/amogh/Stuff/CMU/datasets/DISFA_data/ActionUnit_Labels/
/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/


In [27]:
import mnist_helpers

Helper Functions

## FAU4_1

Loading the saved features:

In [28]:
dir_features_hog_1_fau4_1=DISFA_path+"/features/hog_1/fau4_1/"
dir_features_hog_1_fau4_1

'/media/amogh/Stuff/CMU/datasets/DISFA_data//features/hog_1/fau4_1/'

In [29]:
features_fau4_1_hog_1_positives=bcolz.carray(rootdir=dir_features_hog_1_fau4_1+'positives/',mode='r')
print("features_fau4_1_hog_1_positives loaded, shape is: ",features_fau4_1_hog_1_positives.shape)

features_fau4_1_hog_1_positives loaded, shape is:  (1874, 448)


In [30]:
features_fau4_1_hog_1_negatives=bcolz.carray(rootdir=dir_features_hog_1_fau4_1+'negatives/',mode='r')
print("features_fau4_1_hog_1_negatives loaded, shape is : ",features_fau4_1_hog_1_negatives.shape)

features_fau4_1_hog_1_negatives loaded, shape is :  (1997, 448)


Defining the training and testing data

In [31]:
X=np.concatenate([features_fau4_1_hog_1_positives,features_fau4_1_hog_1_negatives[:1874]])
X.shape

(3748, 448)

In [32]:
Y=np.concatenate([np.repeat(1,features_fau4_1_hog_1_positives.shape[0]),np.repeat(0,features_fau4_1_hog_1_positives.shape[0])])
Y.shape

(3748,)

In [33]:
X_train,X_test,y_train,y_test=sklearn.model_selection.train_test_split(X,Y,test_size=0.15,random_state=42)

Defining the parameter range for SVM:

In [17]:
gamma_range=np.outer(np.logspace(-3,0,4),np.array([1,5]))
gamma_range=gamma_range.flatten()
C_range=np.outer(np.logspace(-1,1,3),np.array([1,5]))
C_range=C_range.flatten()
parameters={'kernel':['rbf'],'C':C_range,'gamma':gamma_range}

In [18]:
svm_clsf=svm.SVC()
grid_clsf=sklearn.model_selection.GridSearchCV(estimator=svm_clsf,param_grid=parameters,n_jobs=1,verbose=2)

Start training and parameter search 

In [19]:
start_time=dt.datetime.now()
print('Start param searching at {}'.format(str(start_time)))
grid_clsf.fit(X_train,y_train)
elapsed_time=dt.datetime.now()-start_time
print('Elapsed time, param searching {}'.format(str(elapsed_time)))
sorted(grid_clsf.cv_results_.keys())

Start param searching at 2018-05-28 15:21:16.675393
Fitting 3 folds for each of 48 candidates, totalling 144 fits
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] ................... C=0.1, gamma=0.001, kernel=rbf, total=   3.8s
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.2s remaining:    0.0s


[CV] ................... C=0.1, gamma=0.001, kernel=rbf, total=   3.7s
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] ................... C=0.1, gamma=0.001, kernel=rbf, total=   3.8s
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] ................... C=0.1, gamma=0.005, kernel=rbf, total=   3.7s
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] ................... C=0.1, gamma=0.005, kernel=rbf, total=   3.7s
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] ................... C=0.1, gamma=0.005, kernel=rbf, total=   3.7s
[CV] C=0.1, gamma=0.01, kernel=rbf ...................................
[CV] .................... C=0.1, gamma=0.01, kernel=rbf, total=   3.6s
[CV] C=0.1, gamma=0.01, kernel=rbf ...................................
[CV] .................... C=0.1, gamma=0.01, kernel=rbf, total=   3.6s
[CV] C=0.1, gamma=0.01, kernel=rbf ...................................
[CV] .

[Parallel(n_jobs=1)]: Done 144 out of 144 | elapsed: 10.4min finished


Elapsed time, param searching 0:10:29.867128


['mean_fit_time',
 'mean_score_time',
 'mean_test_score',
 'mean_train_score',
 'param_C',
 'param_gamma',
 'param_kernel',
 'params',
 'rank_test_score',
 'split0_test_score',
 'split0_train_score',
 'split1_test_score',
 'split1_train_score',
 'split2_test_score',
 'split2_train_score',
 'std_fit_time',
 'std_score_time',
 'std_test_score',
 'std_train_score']

In [20]:
classifier=grid_clsf.best_estimator_
params=grid_clsf.best_params_

scores=grid_clsf.cv_results_['mean_test_score'].reshape(len(C_range),len(gamma_range))
# plot_param_space_scores(scores,C_range, gamma_range)

In [21]:
scores

array([[0.50643642, 0.58587127, 0.72056515, 0.78806907, 0.79717425,
        0.80847724, 0.77488226, 0.50643642],
       [0.58587127, 0.77896389, 0.79215071, 0.85808477, 0.8822606 ,
        0.91020408, 0.89199372, 0.50643642],
       [0.74474097, 0.79403454, 0.8166405 , 0.89387755, 0.91302983,
        0.93186813, 0.9255887 , 0.62323391],
       [0.7956044 , 0.87535322, 0.8989011 , 0.92621664, 0.93783359,
        0.94128728, 0.93406593, 0.66436421],
       [0.82009419, 0.89858713, 0.91145997, 0.93406593, 0.93814757,
        0.94128728, 0.93406593, 0.66436421],
       [0.89795918, 0.91459969, 0.92182104, 0.92841444, 0.93877551,
        0.94128728, 0.93406593, 0.66436421]])

In [22]:
expected = y_test
predicted = classifier.predict(X_test)
print(predicted)
print("predicted and expected are same in: ", np.sum(expected==predicted)," out of ", expected.shape[0])

print("Classification report for classifier %s:\n%s\n"
      % (classifier, metrics.classification_report(expected, predicted)))
      
cm = metrics.confusion_matrix(expected, predicted)
print("Confusion matrix:\n%s" % cm)

# plot_confusion_matrix(cm)

print("Accuracy={}".format(metrics.accuracy_score(expected, predicted)))


[0 0 1 1 0 0 1 1 0 1 1 1 1 1 1 0 1 1 0 0 0 0 1 0 1 1 1 1 0 1 1 1 1 0 1 1 1
 0 1 0 0 0 0 1 1 0 0 1 1 0 0 1 1 1 0 0 0 1 0 1 1 0 1 0 0 0 0 1 0 0 1 1 1 0
 1 0 1 1 0 1 0 1 0 1 0 1 0 0 1 0 1 1 1 1 0 1 1 0 1 1 1 0 0 1 0 1 0 1 1 1 1
 1 0 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 0 0 0 1 1 1 1 0 1 1 1 1 0 1 0 1
 1 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 0 0 0 0 1 1 0 0 0 0 0 0 1 1 0 1 1 1
 0 1 0 0 0 1 1 1 1 1 1 0 0 0 0 0 1 1 1 1 1 0 0 1 1 0 1 0 0 0 1 0 1 0 0 0 0
 1 0 0 0 1 1 1 1 1 0 0 0 1 1 0 1 0 0 0 0 1 1 1 0 0 1 1 0 1 0 1 1 1 0 0 0 0
 0 1 0 1 1 1 0 1 0 0 0 0 0 0 1 1 1 0 1 0 1 1 1 0 1 1 0 0 1 1 1 0 1 0 0 1 0
 1 1 1 0 0 1 1 0 0 1 0 0 1 1 0 1 1 0 0 1 1 0 1 1 1 1 0 1 0 1 1 0 0 1 1 0 1
 1 1 1 1 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 0 1 0 1 1 1 1 1 1 0 0 0 0 0 1 1 0 1
 0 0 1 1 1 0 0 1 1 0 0 0 1 1 0 0 0 0 1 1 0 0 0 1 0 0 0 1 1 1 0 1 0 0 1 0 1
 1 1 1 0 0 0 0 1 0 0 1 0 1 0 1 0 1 1 1 0 0 0 0 0 0 0 1 1 1 0 1 1 1 1 0 1 0
 0 1 0 1 1 1 1 0 1 1 0 0 1 0 0 0 0 0 0 1 1 1 0 1 1 0 0 0 1 1 0 1 1 0 1 1 0
 1 1 1 1 1 1 0 1 1 1 1 0 

**Saving the model to disk**

In [24]:
DISFA_path

'/media/amogh/Stuff/CMU/datasets/DISFA_data/'

In [16]:
models_path=DISFA_path+'models/'
if not os.path.exists(models_path):
    os.makedirs(models_path)

In [39]:
with open(models_path+'fau4_1_hog_1.pickle','wb') as handle:
    pickle.dump(classifier,handle)

**Loading a model from disk**

In [25]:
with open(models_path+'fau4_1_hog_1.pickle','rb') as handle:
    clf1=pickle.load(handle)

**Calculating features for a cropped image from disk**

In [18]:
def calcHOGfeatures(cropped_im_path):
    #instead of 448, put the dimension of the HOG vector
    features_fau4_1_hog_1_negatives=np.empty((0,448))
    im=cv2.imread(cropped_im_path,0)
    fd, hog_image = hog(im, orientations=8, pixels_per_cell=(8,8),cells_per_block=(1, 1), visualise=True)
    print ("The shape of HOG features vector is: ",features_fau4_1_hog_1_negatives.shape)
    return fd

In [23]:
dir_cropped_FAU4_1=DISFA_path+"/features/cropped_images/FAU4_1/"
print(dir_cropped_FAU4_1)
dir_cropped_FAU4_1_positives=dir_cropped_FAU4_1+"positives/"
print(dir_cropped_FAU4_1_positives)
dir_cropped_FAU4_1_negatives=dir_cropped_FAU4_1+"negatives/"
print(dir_cropped_FAU4_1_negatives)

/media/amogh/Stuff/CMU/datasets/DISFA_data//features/cropped_images/FAU4_1/
/media/amogh/Stuff/CMU/datasets/DISFA_data//features/cropped_images/FAU4_1/positives/
/media/amogh/Stuff/CMU/datasets/DISFA_data//features/cropped_images/FAU4_1/negatives/


In [24]:
list_cropped_fau4_1_positive_files=os.listdir(dir_cropped_FAU4_1_positives)
list_cropped_fau4_1_positive_files[0]

'RightVideoSN001_1340.png'

In [ ]:
calcHOGfeatures(list_cropped_fau4_1_positive_files)

In [46]:
clf1.predict(features_fau4_1_hog_1_positives)

array([1, 1, 1, ..., 1, 1, 1])

In [44]:
classifier.predict(features_fau4_1_hog_1_positives)

array([1, 1, 1, ..., 1, 1, 1])

In [38]:
import dlib

In [39]:
detector=dlib.get_frontal_face_detector()
predictor=dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [40]:
def detectFace(impath):
    # load the input image, resize it, and convert it to grayscale
    image = cv2.imread(impath)
#     image = imutils.resize(image, width=500)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # detect faces in the grayscale image
    rects = detector(gray, 1)

    # loop over the face detections
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
#         print(shape.shape)
        # convert dlib's rectangle to a OpenCV-style bounding box
        # [i.e., (x, y, w, h)], then draw the face bounding box
        (x, y, w, h) = face_utils.rect_to_bb(rect)
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 10)
        # show the face number
        cv2.putText(image, "Face #{}".format(i + 1), (x - 10, y - 10),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        basePath=impath.split('.')[0]
#         uncomment lines to write the landmarks to a text file
        lmpath=basePath+"_landmarks"+str(i+1)+".txt"
        text_file = open(lmpath, "w")
        for (x, y) in shape:
#             print x,y
            cv2.circle(image, (x, y), 10, (0, 0, 255), -1)
            text_file.write(str(x)+' '+str(y)+'\n')
        text_file.close()
        
    # cv2.imshow("im", image)
    # cv2.waitKey(1000)
#     print 'These are the detected faces and the landmarks, which have been written to : ', lmpath
#     print imageTouse.shape,
#     print 'landmarks written to: ', lmpath
    plt.imshow(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))
#         plt.imshow(image)
    return lmpath

In [41]:
import cv2

ModuleNotFoundError: No module named 'cv2'